# Lec 3: Object Detection 
Let's talk object detection now. The first approach was using sliding 2D CNNs. Let's consider a rectangle of predefined dimensions. We slide through the given image, and each resulting snapshot will be passed to a CNN to determine whether there is an object there or not. This model, as its explanation sounds is quite simple. Thus, it comes with a number of issues:
1. What if an object in the image is improportionally large / small in comparison to the detection box
2. The model is definitely not efficient

* The basic model was improved further by first considering a smaller copy of the given image. Performing the same process. If the results are positive (an object is there), then the image is resized to larger dimensions. The initial procedure is only performed in regions where the an object detected in smaller copies. The algorithm's success is quite dependent on the efficiency of the first step.

## Selective Search
It is an algorithm that detects interesting objects based on a number of heuristics. The boxes of these interesting objects are later passed to a CNN.

## Intersection Over Union
Assuming we have the ground truth bounding boxes, we consider the boxes predicted by the model. One metric to evaluate the performance is called Intersection Over Union. The main idea is to calculate the ratio of I over U where:
* I is the area of the intersection of the two boxes
* U is the area covered by the union of the two boxes.



## Non-maximum supression
* One major issue about selective search empowered algorithms is reporting parts of what we (humans) consider an atomic object as stand alone objects: the leg / head of a cat. Therefore, selective search would return multiple suggestions. Nevertheless IOU would generally evaluate these individual parts with certain high values that cannot be just ignored.
*  WELL, you probably guessed it, we need to come up with some filtering process, right ?. The main algorithm is referred to as non maximum supression. Let me describe it in my own words:
1. first consider the proposals: sort them according to a certain metric $S$ and put them in a list, say $B$. Take the one with the highest metric
2. The current object selected is compared to every object in $B$
3. Assuming a IOU treshold is already defined $N_T$, then any object with IOU(current best object, object) $ \geq N_T$ will be removed from $B$ or using the technical term: suppressed
4. Repeat the process until $B$ is empty.  

The algorithm is quadratic with respect to the number of initial proposals and linear at best. Besides performance, The NMS algorithm introduces additional issues: The choice of $N_T$ is quite critical, and might suppress legitimate and quite objectively good objects in a crawded image. As a solution for the 2nd issue, the soft-NMS was proposed.  

## RCNN
RCNN: an abbreviation of Regions with CNN features. It is based on selective search where the latter extracts around 2k regions. These regions are then evaluated by a CNN.
This architecture is quite problematic for a number of reasons: 
1. Evaluting 2000k classifications per single image
2. very slow performanace
3. selective search is based on heuristics (a static algorithm). Such algorithm might not be effective for all data problems

## Fast RCNN
The main twick with FRCNN is first applying convolution layers (no FC ones) and then extracting the different regions from the resulting image. The proposals are then passed to the fully connected part of the architecture.

The image resulting from the convolution operations is referred to as the feature map. Selective search produces cuts of different shapes. They cannot be fed directly to the Fully connected layers as they require uniform dimensions. The solution is ROI pooling which is an abbreviation of Region Of Interest Pooling. The latter incorporates Max Pooling in a mechanism that guarantees uniform output shape regardless of the input shape.

One important note is that selective search cannot be applied directory on a feature map. Well, we can't just sit down and cross our arms. We still apply the algorithm on the original image. Nevertheless, using the ratio between the feature map shape and the original's to map the coordinates of the interesting regions to their correspondings in the feature map.

### Performance analysis
When Comparing the performance of RCNN and Fast RCNN, we can see that selective search takes around $2s$ out of almost $50s$ in inference time. The bottle neck in RCNN is clearly the classification of $2000$ regions proposed by selective search. Nevertheless, The same $2s$ in Fast RCNN represent a more than $85\%$ of inference time. Therefore, an even faster performance requires a more efficient algorithm than selective search. 


# Lec 4: Object Detection
## Faster RCNN
Selective search is relatively slow: it represents the performance bottleneck in the faster RCNN model. Faster RCNN abandons this approach completely.
1. We consider a CNN backbone which is mainly a deep stack of combinations of convolutional layers and maxpooling layers.
2. The given images is passed by this backbone to produced a feature map
3. The model does not follow a serial architecture as the feature map is further used to determine the regions proposals.
4. Faster RCNN defines anchors: which are boxes used to modify the images. Assuming $K$ anchors, A predefined $n * n$ slides through the feature map. For each window, $K$ regions are passed to the next layer. The latter passed further down the architecture.
5. Each region is first associated an object score: if this region is classied as an object it is further passed down the architecture to be classified to the actual concrete classes

# Lec 4: Object Detection
## Faster RCNN
As explained above the selective search algorithm is the bottleneck in the Faster RCNN model. The replacement is as follows:
1. Passing the original image through the so-called CNN BACKBONE: which is a block of convolutional layers and max pooling. The latter converts the image to a feature map with smaller dimension while significantly increasing the number of ***channels***.
2. 2 different processes are applied on the feature map. The first one is a set of $2k$ convolutional filters / kernels of shape $(1, 1, n\_c)$. The second is a set of $4k$ filters with shape $1, 1, n\_c$. The first one is for classification. The second is for bound box regression. and $k$ is the number of anchors.
3. The classification at this stage of the model does not consider the concrete classes of the problem at hand but different object-related classes. Something in the vacinity of:
    * object
    * background
    * around an object
4. Only the pairs of (class, bound box) with a positive class are passed further down the architecture. The bounding box are first proposed by anchors, then those anchors boxes are improved with a FC regression layer. Those improved versions are passed to other FC layers for the final predictions. 

# Auto Encoder and shit

* Auto encoders: the encoder is basically a feed forward neural network that projects the initial image to a latent space and build a compact representation rich of information about input. Generally the best, the richest embedding is the one created by the last convolutional layer
* The decoder does the exact opposite: convert the latent space representation to the initial image
* This training is unsupervised, the loss is generally regression where the initial input is compared to the output generated by the decoder.
* decoder uses upscaling + convolution operators
* Based on the same principle, idea, we can use this architecture to denoise images
* the main idea about large auto-encoders is using the same small auto-encoder through a large number of intermediate steps
* Variational autoencoders output vectors that represent distributions: mean and std. Much more powerful and general. They are more robust to slight changes in the input
* semantic segmentation: is detect every single non-background class: cares will be together, persons together
* instance segmentation: each single instance of each possible class is segmented separately
* we can have output of height * width * num_classes. Each channel will represent background (not the class in question) and forground the class in question. The main idea is here is that each class should be segmented which would enable the model. The training is much more demanding in terms of time and resources. Nevertheless, with the right loss function it can produce quite powerful results.
* the difference in size in the Fully Convolutional NN refers to the ration between the size of the input and that of the output mask: the smaller the stride the more detailed the segmentation is
* batch normalization is needed to keep the gradients under control.
* 
* UNet solves the problem of the model forgetting about the finest details of the initial input
* The idea is not too difficult: The encoder block is made out of convolutional blocks, each one ends up reducing the size of the output. Nevertheless these intermediat outputs are fed to equivalent counterparts in the decoder.
